# What is about 

Here is analysis of the cell cycle based on the single cell RNA sequencing data from Kaggle competition: Multimodal Single-Cell Integration Competition. (Bone marrow cells - mostly progentors. More precisely: mobilized peripheral CD34+ hematopoietic stem and progenitor cells (HSPCs) isolated from four healthy human donors). 


(с) A.Chervov
    
See paper: [https://arxiv.org/abs/2208.05229](https://arxiv.org/abs/2208.05229) for cell cycle analysis.
"Computational challenges of cell cycle analysis using single cell transcriptomics" Alexander Chervov, Andrei Zinovyev.     All the work for that paper has been done on Kaggle - see datasets and code therein:
    [https://www.kaggle.com/alexandervc/datasets?scroll=true](https://www.kaggle.com/alexandervc/datasets?scroll=true)  [https://www.kaggle.com/andreizinovyev](https://www.kaggle.com/andreizinovyev)    Hundreds of single cell RNA seq datasets were analyzed. 
    
    
#### Remarks:

At that script we look on  'MkP': 'Megakaryocyte Progenitor' -   10740 cells. (Afraid of RAM crash).

See also discussion topic: https://www.kaggle.com/competitions/open-problems-multimodal/discussion/350314


#### Conclusions: 

    The cell cycle pattern is changing day by day. 
    Day 2 - active proliferation, no G0 part. 
    Day 3 - G0 part appears
    Day 4 - G0 part almost DISappears (that is strange, may be incorrect cell type identification )
    Day 7 - "standard"(big triangle with G0) pattern appear, G0 appear, but "fast"-like part is still present
    SigFast Figures works nicely - for day 7 we can see 2 cell cycles - fast and standard 
    
    By the previous analysis scripts  it became clear that for better analysis of the cell cycle for the current dataset 
    we need to 1) split by cell types 2) make denoisings 
    Script (denoising analysis) 2: https://www.kaggle.com/code/alexandervc/mmscel-cell-cycle-02-with-denoising
    Script (various splits by cell types, donors, days,  etc) 1: https://www.kaggle.com/code/alexandervc/mmscel-cell-cycle-01-preliminary-draft
    

Cells are "mobilized peripheral CD34+ " - here "MOBILIZED" seems to mean that donors were given special drugs for weeks before the experiment such that their bone marrow cells move to the blood. It seems it is usually achieved by the proliferation stimulation. That is why we may expect stronger proliferation - so it might be different from other datasets with similar cells (typically there are datasets WITHOUT" mobilization").

So active proliferation at the begining and fading later is consitent with the information above.
What is strange that on day 3 - there is more fading of prolifetation , than day 4. But it might be technical reason  due to incorrect cell type identification. 


#### Technicality

    We use preprocessed "denoised" scRNA-seq data - we use our own "pooling" (simple and fast) with n_neigbour = 20
    The preprocessing script: https://www.kaggle.com/competitions/open-problems-multimodal
    It requires huge RAM so the Saturn cloud was used to run it. 
    
    The huge h5ad would take 12G out 16 Kaggle memory - so can do nothing in downstream, 
    So we back it on disk, but extract part corresponding the particular cell type




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install/import and auxilliary functions 

In [ ]:
!pip install scanpy
import scanpy as sc
import anndata

import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
#plt.style.use('dark_background')
import seaborn as sns

dict_cell_types = { 'MasP': 'Mast Cell Progenitor',
'MkP': 'Megakaryocyte Progenitor',
'NeuP': 'Neutrophil Progenitor',
'MoP': 'Monocyte Progenitor',
'EryP': 'Erythrocyte Progenitor',
'HSC': 'Hematoploetic Stem Cell',
'BP': 'B-Cell Progenitor'}


G1S_genes_Tirosh = ['ENSMUSG00000005410', 'ENSMUSG00000027342', 'ENSMUSG00000025747', 'ENSMUSG00000024742', 'ENSMUSG00000002870', 'ENSMUSG00000022673', 'ENSMUSG00000030978', 'ENSMUSG00000029591', 'ENSMUSG00000031821', 'ENSMUSG00000026355', 'ENSMUSG00000055612', 'ENSMUSG00000037474', 'ENSMUSG00000025395', 'ENSMUSG00000001228', 'ENSMUSG00000031629', 'ENSMUSG00000025001', 'ENSMUSG00000023104', 'ENSMUSG00000028884', 'ENSMUSG00000028693', 'ENSMUSG00000030346', 'ENSMUSG00000006715', 'ENSMUSG00000027242', 'ENSMUSG00000004642', 'ENSMUSG00000028212', 'ENSMUSG00000041712', 'ENSMUSG00000030726', 'ENSMUSG00000024151', 'ENSMUSG00000022360', 'ENSMUSG00000027323', 'ENSMUSG00000020649', 'ENSMUSG00000000028', 'ENSMUSG00000017499', 'ENSMUSG00000039748', 'ENSMUSG00000032397', 'ENSMUSG00000022422', 'ENSMUSG00000030528', 'ENSMUSG00000028282', 'ENSMUSG00000028560', 'ENSMUSG00000042489', 'ENSMUSG00000006678', 'ENSMUSG00000022945', 'ENSMUSG00000034329', 'ENSMUSG00000046179']
G2M_genes_Tirosh = ['ENSMUSG00000054717', 'ENSMUSG00000019942', 'ENSMUSG00000027306', 'ENSMUSG00000001403', 'ENSMUSG00000017716', 'ENSMUSG00000027469', 'ENSMUSG00000020914', 'ENSMUSG00000024056', 'ENSMUSG00000062248', 'ENSMUSG00000026683', 'ENSMUSG00000028044', 'ENSMUSG00000031004', 'ENSMUSG00000019961', 'ENSMUSG00000026605', 'ENSMUSG00000037313', 'ENSMUSG00000020808', 'ENSMUSG00000034349', 'ENSMUSG00000032218', 'ENSMUSG00000048327', 'ENSMUSG00000037725', 'ENSMUSG00000020897', 'ENSMUSG00000027379', 'ENSMUSG00000012443', 'ENSMUSG00000015749', 'ENSMUSG00000036752', 'ENSMUSG00000022385', 'ENSMUSG00000024795', 'ENSMUSG00000044783', 'ENSMUSG00000023505', 'ENSMUSG00000020737', 'ENSMUSG00000006398', 'ENSMUSG00000038379', 'ENSMUSG00000044201', 'ENSMUSG00000028678', 'ENSMUSG00000022391', 'ENSMUSG00000038252', 'ENSMUSG00000037544', 'ENSMUSG00000048922', 'ENSMUSG00000028873', 'ENSMUSG00000027699', 'ENSMUSG00000032254', 'ENSMUSG00000020330', 'ENSMUSG00000027496', 'ENSMUSG00000068744', 'ENSMUSG00000036777', 'ENSMUSG00000004880', 'ENSMUSG00000040549', 'ENSMUSG00000045328', 'ENSMUSG00000005698', 'ENSMUSG00000026622', 'ENSMUSG00000035293', 'ENSMUSG00000074802', 'ENSMUSG00000009575', 'ENSMUSG00000029177']

G1S_genes_Tirosh = ['ENSG00000100297', 'ENSG00000132646', 'ENSG00000176890', 'ENSG00000168496', 'ENSG00000073111', 'ENSG00000104738', 'ENSG00000167325', 'ENSG00000076248', 'ENSG00000131153', 'ENSG00000076003', 'ENSG00000144354', 'ENSG00000143476', 'ENSG00000198056', 'ENSG00000276043', 'ENSG00000151725', 'ENSG00000119969', 'ENSG00000049541', 'ENSG00000117748', 'ENSG00000132780', 'ENSG00000111247', 'ENSG00000112312', 'ENSG00000092470', 'ENSG00000163950', 'ENSG00000175305', 'ENSG00000012963', 'ENSG00000077514', 'ENSG00000095002', 'ENSG00000156802', 'ENSG00000051180', 'ENSG00000171848', 'ENSG00000093009', 'ENSG00000094804', 'ENSG00000174371', 'ENSG00000075131', 'ENSG00000136982', 'ENSG00000197299', 'ENSG00000118412', 'ENSG00000162607', 'ENSG00000092853', 'ENSG00000101868', 'ENSG00000159259', 'ENSG00000136492', 'ENSG00000129173']
G2M_genes_Tirosh = ['ENSG00000164104', 'ENSG00000170312', 'ENSG00000137804', 'ENSG00000175063', 'ENSG00000089685', 'ENSG00000088325', 'ENSG00000131747', 'ENSG00000080986', 'ENSG00000123975', 'ENSG00000143228', 'ENSG00000173207', 'ENSG00000148773', 'ENSG00000120802', 'ENSG00000117724', 'ENSG00000013810', 'ENSG00000129195', 'ENSG00000113810', 'ENSG00000157456', 'ENSG00000169607', 'ENSG00000136108', 'ENSG00000178999', 'ENSG00000169679', 'ENSG00000138160', 'ENSG00000143401', 'ENSG00000188229', 'ENSG00000075218', 'ENSG00000138182', 'ENSG00000123485', 'ENSG00000111665', 'ENSG00000189159', 'ENSG00000117399', 'ENSG00000112742', 'ENSG00000158402', 'ENSG00000142945', 'ENSG00000100401', 'ENSG00000010292', 'ENSG00000126787', 'ENSG00000184661', 'ENSG00000134690', 'ENSG00000114346', 'ENSG00000137807', 'ENSG00000072571', 'ENSG00000087586', 'ENSG00000134222', 'ENSG00000011426', 'ENSG00000143815', 'ENSG00000175216', 'ENSG00000138778', 'ENSG00000102974', 'ENSG00000117650', 'ENSG00000092140', 'ENSG00000139354', 'ENSG00000094916', 'ENSG00000115163']
list_genes_fastCCsign = ['ENSG00000170312', 'ENSG00000175063', 'ENSG00000131747', 'ENSG00000120802', 'ENSG00000123485', 'ENSG00000167325', 'ENSG00000111247', 'ENSG00000171848', 'ENSG00000093009', 'ENSG00000197299', 'ENSG00000136492', 'ENSG00000129173', 'ENSG00000184260']

G1S_genes_Tirosh = ['MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MLF1IP', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8']
G2M_genes_Tirosh = ['HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A', 'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'HN1', 'CDC20', 'TTK', 'CDC25C', 'KIF2C', 'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA']
list_genes_fastCCsign = ['CDK1', 'UBE2C', 'TOP2A', 'TMPO', 'HJURP', 'RRM1', 'RAD51AP1', 'RRM2', 'CDC45', 'BLM', 'BRIP1', 'E2F8', 'HIST2H2AC']


def phase_plot(adata, list_genes1, list_genes2, mask = None, list_color_by = [None], str_suptitle = None, 
               str_title = None,
               xfigsize = 20, yfigsize = 10,
               xlabel = 'G1S score', ylabel = 'G2M score',
               n_x_subplots = 1, n_y_subplots = 1 #  #len(list_selected_cell_lines)
              ):

    if mask is None:
        mask = np.ones( adata.X.shape[0]).astype(bool) #  

    #list_color_by = ['total_counts']# , 'Cell cycle phase']

    
    total_count = 0
    for color_by_mode in  list_color_by: # , 'pct_counts_mt']:
        
        if n_x_subplots * n_y_subplots <= 1:
            fig = plt.figure(figsize = (xfigsize,yfigsize) ); c = 0
            if str_suptitle is not None:
                plt.suptitle(str_suptitle + ' N_cells= ' +str(mask.sum() ) + ' ' , fontsize = 20 )#' 
        else:
            if ( total_count % (n_x_subplots * n_y_subplots) ) == 0: 
                fig = plt.figure(figsize = (xfigsize,yfigsize) ); c = 0
                if str_suptitle is not None:
                    plt.suptitle(str_suptitle + ' N_cells= ' +str(mask.sum() ) + ' ' , fontsize = 20 )#' 
            c+=1; fig.add_subplot(n_y_subplots,n_x_subplots,c);
            
        total_count += 1
        
        
        color_by_field_name = color_by_mode
        color_by = None
        if color_by_mode is None:
            pass
        elif color_by_mode in adata.var.index:
            
            color_by = pd.Series( np.asarray(adata[mask,color_by_mode].X.sum(axis=1) ).ravel() , name = color_by_mode )
        elif color_by_mode in adata.obs:
            color_by = (adata[mask].obs[color_by_field_name])
        elif 'median_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode[16:]
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > np.median( (adata[mask].obs[color_by_field_name]) )
        elif 'threshold_binarize_' in color_by_mode:
            color_by_field_name = color_by_mode.split('_')[3]
            threshold_binarize = float( color_by_mode.split('_')[2] )
            if color_by_field_name in adata.obs:
                color_by = (adata[mask].obs[color_by_field_name]) > threshold_binarize
        else:
            color_by = None

        ll = list_genes1
        I = np.where(adata[mask].var.index.isin(ll) > 0 )[0] # .sum()
        v1 = np.asarray(adata[mask].X[:,I].mean(axis = 1)).ravel()
        #print(len(I), v1.mean(), len(v1))    
        
        ll = list_genes2#  G2M_genes_Tirosh
        I = np.where(adata[mask].var.index.isin(ll) > 0 )[0] # .sum()
        #print(len(I))
        v2 = np.asarray(adata[mask].X[:,I].mean(axis = 1)).ravel()    


        if str_title is not None:
            plt.title( str_title  , fontsize = 20 )
        if color_by is None:
            ax = sns.scatterplot(x=v1, y = v2)# ,  hue= color_by,   alpha = 0.8, marker = '.')#, legend=None)
        else:
            #color_by = (adata.obs[color_by_field_name]) > np.median( adata.obs[color_by_field_name].values ) 
            ax = sns.scatterplot(x=v1, y = v2,  hue= color_by, palette = "rainbow")# 'viridis")# sns.color_palette("viridis", as_cmap=True),
                                #)# ,   alpha = 0.8, marker = '.')#, )#, legend=None)
            plt.setp(ax.get_legend().get_texts(), fontsize='20') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='20') # for legend title


        #plt.title(str_inf)# str_data_inf + ' ' + str_reducer + str_genes_inf + ' ' + col)
        plt.xlabel(xlabel , fontsize = 20)
        plt.ylabel(ylabel, fontsize = 20 )
    plt.show()         
        

# Load data and extract only one cell type - Megakaryocyte Progenitor

In [ ]:
#fn = '/kaggle/input/scrnaseq-tabula-sapiens-human-part-2/TS_epithelial.h5ad'
fn = '/kaggle/input/multimodal-singlecell-integration-related-data-01/cite_seq_inputs.h5ad'
fn = '/kaggle/input/multimodal-singlecell-integration-related-data-01/cite_seq_inputs_denoising_pooling_n_neighbours20.h5ad'
str_data_inf = ' CITEseq '
print(str_data_inf, fn )
t0 = time.time()
adata = sc.read(fn,  backed='r' )
print(np.round(time.time()-t0,0),'second passed on loading')
adata# 

In [ ]:
print('820 M RAM consumed - quite little')

In [ ]:
print( adata.obs['cell_type'].value_counts() )
print( adata.obs['Train0Test1'].value_counts() )
print( adata.obs['day'].value_counts() )
print( adata.obs['donor'].value_counts() )
print( adata.obs['Train0Test1'].value_counts() )

In [ ]:
import time
t0 = time.time()
col = 'cell_type'
uv = 'MkP'#  'MoP'
mask = adata.obs[col] == uv
print(mask.sum())

a = adata[mask,:].to_memory()# Otherwise cannot change like: a.uns['Cell type'] = cell_type_loc
print(('%.f seconds passsed'%(time.time()-t0)))
str_data_inf = ' CITEseq  ' + uv + ' ' +dict_cell_types[uv]   + ' n_neighbours='+str(20)#n_neighbours)
print( str_data_inf )
a 

In [ ]:
a.obs['str_donor'] = a.obs['donor'].apply(lambda x: str(x))

# Day by day big figures

In [ ]:
col = 'day'
for uv in a.obs[col].unique():
    mask = a.obs[col] == uv
    print(mask.sum())
    phase_plot(a, list_genes1 =G1S_genes_Tirosh , list_genes2=G2M_genes_Tirosh, 
               mask = mask,
               str_title =  ' G1/S-G2/M plot.   Tirosh genes ' + col + ' = ' + str(uv)  ,
               str_suptitle = str_data_inf, #  + ' n_cells = ',# + str( adata.shape[0] ),
                n_x_subplots = 1, n_y_subplots = 1, 
          list_color_by = [  'str_donor' ],) # 'CCNB1','day',  'n_genes_by_counts', 'total_counts', ,'CCNE1'

#  G1/S-G2/M plots to analyse the cell cycle

In [ ]:
mask = np.ones(len(adata))
phase_plot(a, list_genes1 =G1S_genes_Tirosh , list_genes2=G2M_genes_Tirosh, 
           #mask = mask,
           str_title =  ' G1/S-G2/M plot.   Tirosh genes' ,
           str_suptitle = str_data_inf , # + ' n_cells = ',# + str( adata.shape[0] ),
      list_color_by = [ 'CCNB1','day',  'n_genes_by_counts', 'total_counts', 'donor', 'Train0Test1' ],) # ,'CCNE1'

# Day by day 

In [ ]:
col = 'day'
for uv in a.obs[col].unique():
    mask = a.obs[col] == uv
    print(mask.sum())
    phase_plot(a, list_genes1 =G1S_genes_Tirosh , list_genes2=G2M_genes_Tirosh, 
               mask = mask,
               str_title =  ' G1/S-G2/M plot.   Tirosh genes ' + col + ' = ' + str(uv)  ,
               str_suptitle = str_data_inf , # + ' n_cells = ',# + str( adata.shape[0] ),
                n_x_subplots = 2, n_y_subplots = 1, 
          list_color_by = [  'donor', 'Train0Test1' ],) # 'CCNB1','day',  'n_genes_by_counts', 'total_counts', ,'CCNE1'

# Day by day big figure (repeat what was above) 

In [ ]:
col = 'day'
for uv in a.obs[col].unique():
    mask = a.obs[col] == uv
    print(mask.sum())
    phase_plot(a, list_genes1 =G1S_genes_Tirosh , list_genes2=G2M_genes_Tirosh, 
               mask = mask,
               str_title =  ' G1/S-G2/M plot.   Tirosh genes ' + col + ' = ' + str(uv)  ,
               str_suptitle = str_data_inf, #  + ' n_cells = ',# + str( adata.shape[0] ),
                n_x_subplots = 1, n_y_subplots = 1, 
          list_color_by = [  'donor' ],) # 'CCNB1','day',  'n_genes_by_counts', 'total_counts', ,'CCNE1'

# SigFast Figures day by day - quite nice - for day 7 we can see 2 cell cycles - fast and standard 

In [ ]:
col = 'day'
for uv in a.obs[col].unique():
    mask = a.obs[col] == uv
    print(mask.sum())
    phase_plot(a, list_genes1 =list_genes_fastCCsign , list_genes2=G2M_genes_Tirosh, 
               mask = mask,xlabel = 'SigFast score',
               str_title =  ' SigFast-G2/M plot.   Tirosh genes ' + col + ' = ' + str(uv)  ,
               str_suptitle = str_data_inf, #  + ' n_cells = ',# + str( adata.shape[0] ),
                n_x_subplots = 1, n_y_subplots = 1, 
          list_color_by = [  'donor' ],) # 'CCNB1','day',  'n_genes_by_counts', 'total_counts', ,'CCNE1'

#  SigFast Scores Day by day big figure - quite nice

In [ ]:
# list_genes_fastCCsign = ['CDK1', 'UBE2C', 'TOP2A', 'TMPO', 'HJURP', 'RRM1', 'RAD51AP1', 'RRM2', 'CDC45', 'BLM', 'BRIP1', 'E2F8', 'HIST2H2AC']

col = 'day'
for uv in adata.obs[col].unique():
    mask = adata.obs[col] == uv
    print(mask.sum())
    phase_plot(adata, list_genes1 =list_genes_fastCCsign , list_genes2=G2M_genes_Tirosh, 
               mask = mask, xlabel = 'SigFast score',
               str_title =  ' SigFast-G2/M plot.   Tirosh genes ' + col + ' = ' + str(uv)  ,
               str_suptitle = str_data_inf, #  + ' n_cells = ',# + str( adata.shape[0] ),
                n_x_subplots = 1, n_y_subplots = 1, 
          list_color_by = [  'donor' ],) # 'CCNB1','day',  'n_genes_by_counts', 'total_counts', ,'CCNE1'